In [6]:
from gurobipy import *
import numpy as np
import math as mt


In [7]:
nb_contrainte = 7
nb_var = 12

In [8]:
def gen_matrice_utilite(n, p):
        return np.array (
                [
                        [-5, -10, -2, -4, -1, -4, -3, -1, -1, -3, -1, -1],
                        [-3, -4, -6, -2, -3, -6, -1, -2, -4, -5, -1, -1]
                ]
        )

def cout_projet(p):
        return None

def generate_w_prime(n):
        # vecteur w
        w=[]

        # creer le vecteur w'
        w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
        w_prime.append(w[-1])
        return w_prime

def generate_contrainte_matrice(n, p, u):
        ## les variables rk
        coef_rk = []
        for k in range(n):
                for i in range(n):
                        coef_rk.append([])
                        for j in range(n):
                                coef_rk[-1].append(0)
                        coef_rk[-1][k] = 1
        coef_rk = np.array(coef_rk)
        ## les variables bik
        coef_bik = -1 * np.identity(n**2)
        ## les variables x
        coef_xij = []
        # contrainte sur l'existence d'un chemin
        existe_chemin = np.aaray([
                [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [-1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                [0, -1, 0, -1, 0, 0, 1, 1, -1, 0, 0, 0],
                [0, 0, -1, 0, -1, 0, 0, 0, 1, 1, 0, 0],
                [0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 1],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1]
        ])
        
        # concatenation des matrices rk, bik, xk 
        result_concat = np.concatenate(
                (coef_rk, np.concatenate((coef_bik, coef_xij),axis=1)),
                axis=1)

        # ajout de la contrainte sur l'existence d'un chemin entre la source et l'arrivée
        constrainte = np.vstack([result_concat, existe_chemin])
        return constrainte

def second_membre_contraintes(n, budget_limit):
        second_membre =  np.array(
                [0 for i in range(nb_contrainte)]
        )
        second_membre[0] = 1
        second_membre[-1] = -1
        return second_membre

def coef_func_objectif(w_prime, n, p):
        ## coefficient fonction objectif
        c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
        # coef des b_ik
        for i in range(n) :
                for j in range(n) :
                        c.append(-1*w_prime[i])

        c += [0 for i in range(p)] # ajout des coef de x
        c = np.array(c)
        #print(c)
        return c

def declare_variables(m, n, p):
        x = []
        for i in range(n):
                x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
        for i in range (n):
                for j in range(n):
                        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((j+1,(i+1)))))
        for j in range(p):
                x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d" % (j+1) ))
        return x

def definition_func_objectif(m, n, p, c, x):
        obj = LinExpr();
        obj =0
        for j in range(n*n+n+p):
                obj += c[j] * x[j]

        m.setObjective(obj,GRB.MAXIMIZE)
        return obj
        
def definition_contraintes(m, x, constrainte, second_membre, n, p):
        # Definition des contraintes
        # Definition des contraintes
        lignes = range(n*n+1)
        colonnes = range(n*n+n+p)
        #print (lignes, colonnes)
        
        # Attention, certaines sont des egalités
        for i in lignes:
                m.addConstr(quicksum(constrainte[i][j]*x[j] for j in colonnes) <= second_membre[i], "Contrainte%d" % i)
        
def display_vars_value(m):
        print("")                
        print('Solution optimale:')
        mvars = m.getVars()
        names = m.getAttr('VarName', mvars)
        values = m.getAttr('X', mvars)
        result = dict(zip(names, values))
        for key in result:
                print(key,'=',result[key])
        print("")
        print('Valeur de la fonction objectif :', m.objVal)

In [9]:
def experience(n, p):
        matrice_utilite = gen_matrice_utilite(n, p)
        w_prime = generate_w_prime(n)
        contraintes = generate_contrainte_matrice(n, p, matrice_utilite)
        second_membre = second_membre_contraintes(n, sum(couts_projet)/2)
        coef_fonction_objectif = coef_func_objectif(w_prime, n, p)
        m = Model("Question 4.2")
        m.Params.LogToConsole = 0
        x = declare_variables(m, n, p)
        m.update()
        obj = definition_func_objectif(m, n, p, coef_fonction_objectif, x)
        definition_contraintes(m, x, contraintes, second_membre, n, p)

        m.optimize()
        display_vars_value(m)

experience(2, 12)
       

UnboundLocalError: local variable 'matrice_utilite' referenced before assignment